In [1]:
import os
from Bio import SeqIO, Entrez
import pandas as pd

In [2]:

'''
Downloads entries from GenBank Nucleotide by given query
Saves entries to file in GenBank format
Input:
    query - str - query for search in GenBank
Output:
    outfile - str - shortname of output file with sequences in genbank format
'''
def fetch_seq_from_Nucleotide(query, outfile):

    Entrez.email = "A.N.Other@example.com"
    # list with ids obtained by query
    record = Entrez.read(Entrez.esearch(db="nucleotide", term=query, idtype="acc", RetMax=1000000))
    id_list = record['IdList']
    print("Query to GenBank Nucleotide database:\"{}\"".format(query))
    print("Number of records found: {}".format(record["Count"]))
    print("{} records will be downloaded".format(len(id_list)))
    # output file with sequences will be saved in current working dir
    f_name = os.path.abspath(outfile)

    # output file with sequences in fasta format
    file_out = open(f_name, 'w')
    
    # Saving sequences 
    
    i = 0
    # fetch each sequence from GenBank by its ID and writes to ouput file
    for id in id_list:
        i+=1
        if i % 500 ==0:
            print("Downloaded {} sequences".format(i))
        handle = Entrez.efetch(db="nucleotide", id=id, rettype="genbank", retmode="text")
        for line in handle:
            file_out.write(line)
        handle.close()
    file_out.close()
    print('Finished')
    return(f_name)

In [3]:
def fetch_metadata_from_gb(input_file):
    '''
    For each entry in GenBank file retrieves the following data:

    # Identifiers
    Accession
    GenBank title (DEFINITION)

    # Classification
    Organism name
    Species
    Isolate
    Strain
    Family
    Lineage

    #Sequence quality
    Length
    Strand

    Num of annotated CDS

    ORF1a
    ORF1b
    ORF2
    coords
    
    #Sources
    Country
    Tissue/Specimen/Source
    Collection date
    Release date
    Submitters
    
    '''

    '''
     dictionary
     entries_data[GenBank Accession] = {}
     entries_data[GenBank Accession].keys() = [
                                                'Version',
                                                'GenBank title'
                                                'Organism name',
                                                'Species',
                                                'Family',
                                                'Virus Lineage',
                                                'Length',
                                                'Isolate',
                                                'Strain',
                                                'Strand',
                                                'Geo location'
                                                'Country',
                                                'Tissue/Specimen/Source',
                                                'Host',
                                                'Collection date',
                                                'Release date',
                                                'Submitters',
                                                'Sequencing Technology'
                                                'CDS number'
                                                ]
    '''
    entries_data = {}

    source_qualifiers = ["strain", "isolate", "geo_loc_name", "country", "collection_date", "isolation_source", "host"]
    source_qualifiers_columns = ["Strain", "Isolate", "Geo location", "Country", "Collection date", "Tissue/Specimen/Source", "Host"]

    entries = SeqIO.parse(os.path.abspath(input_file), "genbank")

    for entry in entries:
        entries_data[entry.name] = {}
        entries_data[entry.name]['Version'] = entry.id
        entries_data[entry.name]['GenBank title'] = entry.description

        entries_data[entry.name]['Release date'] = entry.annotations['date']
        entries_data[entry.name]['Organism name'] = entry.annotations['organism']
        
        entries_data[entry.name]['Virus Lineage'] = ';'.join(entry.annotations['taxonomy'])
        num_taxa = len(entry.annotations['taxonomy'])
        if num_taxa == 9:
            entries_data[entry.name]['Species'] = entry.annotations['taxonomy'][-1]
        else:
             entries_data[entry.name]['Species'] = 'NA'
        entries_data[entry.name]['Family'] = entry.annotations['taxonomy'][6]
        
        entries_data[entry.name]['Length'] = len(entry)

        for reference in entry.annotations['references']:
            if reference.title == 'Direct Submission':
                entries_data[entry.name]['Submitters'] = reference.authors

        count_cds = 0
        for feature in entry.features:
            if feature.type == 'source':
                for qualif, colname in zip(source_qualifiers, source_qualifiers_columns):
                    if qualif in feature.qualifiers:
                        entries_data[entry.name][colname] = feature.qualifiers[qualif][0]
                    else:
                        entries_data[entry.name][colname] = 'NA'
                if 'environmental_sample' in feature.qualifiers:
                    entries_data[entry.name]['Environmental'] = 'Yes'
                else:
                    entries_data[entry.name]['Environmental'] = 'No'
                    
            if feature.type == 'CDS':
                
                count_cds +=1
                
        entries_data[entry.name]['CDS count'] = count_cds

    meta_dataframe = pd.DataFrame.from_dict(entries_data, orient='index')
    
    meta_dataframe.to_csv(os.path.splitext(os.path.abspath(input_file))[0] + '.csv')
    #country_map - file with abbreviations of countries
    return meta_dataframe


In [4]:
def map_feature(feature, feature_map):
    '''
    feature_map - dictionary, e.g. feature_map['Italia']='ITA'
    feature - possible key from feature_map
    return value if key is in feature_map
    '''
    for k, v in feature_map.items():
        if feature.lower() == k.lower():
            return v
    return feature


def orf_coord(input_file, orf_map):
    '''
    Retrieves the coordinates of ORFs
    
    Input:
        input_file - file with nucleotide sequences in genbank-format
        orf_map - csv file annotation of orfs and their codes
    Output:
        coord_file - file with coordinates
    '''

    # dictionary with possiple names of astrovirus ORFs indicated in note, gene and product qualifiers
    # orf_dict["ORF1a"] = "1A"
    orf_dict = read_csv(orf_map)

    # ORFs which coordinates this script will extract
    orf_types_final = ['1A', '1B', '2']
    # ORFs that can be met
    orf_types = ['1A', '1B', '1AB', '2']

    # name of output file with ORF coordinates
    out_file_name_temp = os.path.splitext(os.path.abspath(input_file))[0]
    out_file_name = out_file_name_temp  + '_orf-coords.csv'
    print(out_file_name)

    # ENTRIES with known errors in annotation
    exceptions_ORF2 = ["MN725025", "KY047739", "S68561", "KP404151","KP404152"] # the last CDS in conflict ORF1b ORF2 is ORF2
    #AB518702 AB518703 is actually RdRp
    exceptions_1a = []

    # dictionaruy with ORF coordinates
    dict_coord = {}

    with open(input_file) as handle:
        records = list(SeqIO.parse(handle, 'gb'))

        # entries with no annotations
        no_annot = 0
        # entries with no annotations of target CDS
        no_annot_target = 0
        # CDS with no annotations of target ORFs
        notarget_annot_CDS = 0
        # CDS with no annotations
        no_annot_CDS = 0

        # Entries with CDS that has annotation "nonstructural protein"
        entries_onlynonstrprotein = []
        entries_onlynonstrprotein_short = []
        entries_conflicting_annotations = []
        entries_noannot = []
        entries_notarget_annot = []
        
        for rec in records:

            if "Patent" in rec.description or "Modified Microbial Nucleic Acid" in rec.description: 
                continue
            
            dict_coord[rec.name] = {}
            # iterate over record features

            # number of CDS
            cds_count = 0
            # number of annotated CDS with target ORFs
            cds_annot_count = 0
            for feature in rec.features:
                
                if feature.type == 'CDS':
                    cds_count += 1
                    
                    # Check codon start
                    if 'codon_start' in feature.qualifiers.keys():
                        cod_start = int(feature.qualifiers['codon_start'][0]) - 1
                        if cod_start<0:
                            print('Codon start < 0, something is wrong')
                    else:
                        cod_start = 0

                    # check all annotations in gene, product, note qualifiers
                    CDS_raw_annotations = []
                    for el in ('product', 'gene', 'note'):
                        CDS_raw_annotations.extend(feature.qualifiers.get(el, []))
                    
                    if len(CDS_raw_annotations) != 0:
                        # now we will leave annotations of target ORFs
                        CDS_annotations = list(map(lambda x: map_feature(x, orf_dict), CDS_raw_annotations))
                        CDS_annotations = list(filter(lambda x: x in orf_types, CDS_annotations))
                        if len(CDS_annotations) == 0:
                            #print("No annotations of target ORFs of CDS in {}".format(rec.name))
                            #print(CDS_raw_annotations)
                            #print(rec.description)
                            notarget_annot_CDS +=1

                            if "nonstructural protein" in CDS_raw_annotations:
                                #print("CDS of {} has the only annotation \'nonstructural protein\' ".format(rec.name))
                                if len(rec) > 300:
                                    entries_onlynonstrprotein.append(rec.name + ':' + rec.description + '\n')
                                else:
                                    entries_onlynonstrprotein_short.append(rec.name + ':' + rec.description + '\n')
                        else:
                            #print("Annotations were successfully found")
                            #print(CDS_annotations)

                            # check for consistency of CDS annotations
                            uniq_annotations = list(set(CDS_annotations))
                            if len(uniq_annotations) == 1:
                                CDS_product = uniq_annotations[0]
                                # CDS is a target ORF, the coordinates are not joined
                                if CDS_product in orf_types_final:
                                    dict_coord[rec.name][CDS_product] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                    dict_coord[rec.name][CDS_product + '-chain'] = feature.location.strand
                                    cds_annot_count +=1
                                elif CDS_product == "1AB":

                                    # check whether the coordinates are joined
                                    if len(feature.location.parts) == 2:
                                        dict_coord[rec.name]['1A'] = [int(feature.location.parts[0].start) + cod_start, int(feature.location.parts[0].end)]
                                        dict_coord[rec.name]['1A-chain'] = feature.location.parts[0].strand
                                        dict_coord[rec.name]['1B'] = [int(feature.location.parts[1].start), int(feature.location.parts[1].end)]
                                        dict_coord[rec.name]['1B-chain'] = feature.location.parts[1].strand
                                        cds_annot_count +=1
                                    elif len(feature.location.parts) == 1:
                                        dict_coord[rec.name]['1B'] = [int(feature.location.parts[0].start) + cod_start, int(feature.location.parts[0].end)]
                                        dict_coord[rec.name]['1B-chain'] = feature.location.parts[0].strand
                                        cds_annot_count +=1
                                    else:
                                        print("strange locations in {}".format(rec.name))
                                        print(feature.location)
                            else:
                                print("Conflicting CDS annotations for {}".format(rec.name))
                                #print(CDS_annotations)
                                #print(uniq_annotations)

                                present_1A = ('1A' in uniq_annotations)
                                present_1B = ('1B' in uniq_annotations)
                                present_1A_1B = present_1A and present_1B
                                present_1AB = ('1AB' in uniq_annotations)
                                present_1A_1AB = present_1A and present_1AB
                                present_1B_1AB = present_1B and present_1AB
                                present_2 = ('2' in uniq_annotations)
                                present_1AB_2 = (present_1A and present_2) or (present_1B and present_2) or (present_1AB and present_2) 
                                
                                
                                if present_1AB_2:
                                    if rec.name in exceptions_ORF2:
                                        dict_coord[rec.name]['2'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                        dict_coord[rec.name]['2-chain'] = feature.location.strand
                                        cds_annot_count +=1
                                        print("Resolved for ORF2")
                                    else:
                                        print(CDS_annotations)
                                        print(uniq_annotations)
                                        print("Could not resolve the conflict")
                                    
                                        entries_conflicting_annotations.append(rec.name + ': ORF1ab + ORF2' + ':' + rec.description + '\n')
                                # If ORF1a and ORF1b are present, we choose ORF1b
                                elif present_1A_1B:
                                    if rec.name in exceptions_1a:
                                        dict_coord[rec.name]['1A'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                        dict_coord[rec.name]['1A-chain'] = feature.location.strand
                                        cds_annot_count +=1
                                        print("Resolved for ORF1a")
                                    else:
                                        dict_coord[rec.name]['1B'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                        dict_coord[rec.name]['1B-chain'] = feature.location.strand
                                        cds_annot_count +=1
                                        entries_conflicting_annotations.append(rec.name + ': ORF1a + ORF1b' + ':' + rec.description + '\n')
                                        print("Resolved for ORF1b")
                                # ORF1ab with joined coordinates
                                elif len(feature.location.parts) == 2:
                                    dict_coord[rec.name]['1A'] = [int(feature.location.parts[0].start) + cod_start, int(feature.location.parts[0].end)]
                                    dict_coord[rec.name]['1A-chain'] = feature.location.parts[0].strand
                                    dict_coord[rec.name]['1B'] = [int(feature.location.parts[1].start), int(feature.location.parts[1].end)]
                                    dict_coord[rec.name]['1B-chain'] = feature.location.parts[1].strand
                                    cds_annot_count +=1
                                    print("Resolved for ORF1ab")
                                elif present_1A_1AB and ('1A' not in dict_coord[rec.name].keys()):
                                    dict_coord[rec.name]['1A'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                    dict_coord[rec.name]['1A-chain'] = feature.location.strand
                                    cds_annot_count +=1
                                    print("Resolved for ORF1a")
                                elif present_1B_1AB and ('1B' not in dict_coord[rec.name].keys()):
                                    dict_coord[rec.name]['1B'] = [int(feature.location.start) + cod_start, int(feature.location.end)]
                                    dict_coord[rec.name]['1B-chain'] = feature.location.strand
                                    cds_annot_count +=1
                                    print("Resolved for ORF1b")
                                elif '1B' in dict_coord[rec.name].keys() or '1A' in dict_coord[rec.name].keys():
                                    print("The coordinates've been extracted from another CDS")

                                
                                    
                        
                    else:
                        #print("No annotations of CDS in {}".format(rec.name))
                        #print(rec.description)
                        #print(feature)
                        no_annot_CDS +=1
            
            if cds_count == 0:
                #print("{} has no CDS in annotation".format(rec.name))
                #print(rec.description)
                no_annot +=1
                entries_noannot.append(rec.name + ':' + rec.description + '\n')
            if cds_annot_count == 0:
                #print("{} has no CDS of target ORFs".format(rec.name))
                #no_annot_target +=1
                #print(rec.description)
                entries_notarget_annot.append(rec.name + ':' + rec.description + '\n')



        # Write extracted coordinates to file
        with open(out_file_name, 'w') as out_file:
            line = "GBAC"
            for ORF in orf_types_final:
                line = line + ',' + ORF + ',' + ORF + '-chain'
            out_file.write(line + '\n')
            for rec_id, values in dict_coord.items():
                s = rec_id
                for orf in orf_types_final:
                    if orf in values:
                        s += f",{values[orf][0]}-{values[orf][1]},{values[orf+'-chain']}"
                    else:
                        s += ",NA-NA,NA"
                s += "\n"
                out_file.write(s)
        out_file.close()


        print("Entries with no CDS annotation: {}".format(no_annot))
        print("Entries with no CDS with target ORF annotation: {}".format(no_annot_target))
        print("CDS with no target ORFs: {}".format(notarget_annot_CDS))
        print("CDS with no annotations: {}".format(no_annot_CDS))

        with open(out_file_name_temp + '_noannotCDS.txt', 'w') as file:
            file.writelines(entries_noannot)

        with open(out_file_name_temp + '_noannot_targetCDS.txt', 'w') as file:
            file.writelines(entries_notarget_annot)
            
        with open(out_file_name_temp + '_noannot_targetCDS.txt', 'w') as file:
            file.writelines(entries_notarget_annot)

        with open(out_file_name_temp + '_conflictingannot.txt', 'w') as file:
            file.writelines(entries_conflicting_annotations)

        with open(out_file_name_temp + '_onlyNSP_long.txt', 'w') as file:
            file.writelines(entries_onlynonstrprotein)
            
        with open(out_file_name_temp + '_onlyNSP_short.txt', 'w') as file:
            file.writelines(entries_onlynonstrprotein_short)

        
        return 0


In [9]:
file_gb = fetch_seq_from_Nucleotide('"txid39733"[Organism]' , "Astroviridae_15102025.gb")

Query to GenBank Nucleotide database:""txid39733"[Organism]"
Number of records found: 16012
16012 records will be downloaded
Downloaded 500 sequences
Downloaded 1000 sequences
Downloaded 1500 sequences
Downloaded 2000 sequences
Downloaded 2500 sequences
Downloaded 3000 sequences
Downloaded 3500 sequences
Downloaded 4000 sequences
Downloaded 4500 sequences
Downloaded 5000 sequences
Downloaded 5500 sequences
Downloaded 6000 sequences
Downloaded 6500 sequences
Downloaded 7000 sequences
Downloaded 7500 sequences
Downloaded 8000 sequences
Downloaded 8500 sequences
Downloaded 9000 sequences
Downloaded 9500 sequences
Downloaded 10000 sequences
Downloaded 10500 sequences
Downloaded 11000 sequences
Downloaded 11500 sequences
Downloaded 12000 sequences
Downloaded 12500 sequences
Downloaded 13000 sequences
Downloaded 13500 sequences
Downloaded 14000 sequences
Downloaded 14500 sequences
Downloaded 15000 sequences
Downloaded 15500 sequences
Downloaded 16000 sequences
Finished


In [5]:
input_file = "Astroviridae_15102025.gb"
data = fetch_metadata_from_gb(input_file)

In [6]:
data

,Version,GenBank title,Release date,Organism name,Virus Lineage,Species,Family,Length,Submitters,Strain,Isolate,Geo location,Country,Collection date,Tissue/Specimen/Source,Host,Environmental,CDS count
PV418556,PV418556.1,Canine astrovirus isolate CaAstV/Dog/AUS/hunte...,08-OCT-2025,Canine astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,6444,"Mifsud,J.C.O., Harvey,E., Van Brussel,K., Olss...",NA,CaAstV/Dog/AUS/huntervalley_7/2022,Australia: Hunter Valley,NA,Aug-2022,feces,Canis lupus familiaris,No,3
PV418555,PV418555.1,Canine astrovirus isolate CaAstV/Dog/AUS/hunte...,08-OCT-2025,Canine astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,6466,"Mifsud,J.C.O., Harvey,E., Van Brussel,K., Olss...",NA,CaAstV/Dog/AUS/huntervalley_6/2022,Australia: Hunter Valley,NA,Aug-2022,feces,Canis lupus familiaris,No,3
PV418548,PV418548.1,Canine astrovirus isolate CaAstV/Dog/AUS/b_hop...,08-OCT-2025,Canine astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,2447,"Mifsud,J.C.O., Harvey,E., Van Brussel,K., Olss...",NA,CaAstV/Dog/AUS/b_hopevale_4/2021,Australia: Hope Vale,NA,Dec-2021,rectal swab,Canis lupus familiaris,No,1
PV418547,PV418547.1,Canine astrovirus isolate CaAstV/Dog/AUS/a_hop...,08-OCT-2025,Canine astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,2416,"Mifsud,J.C.O., Harvey,E., Van Brussel,K., Olss...",NA,CaAstV/Dog/AUS/a_hopevale_4/2021,Australia: Hope Vale,NA,Dec-2021,rectal swab,Canis lupus familiaris,No,1
PV418557,PV418557.1,Canine astrovirus isolate CaAstV/Dog/AUS/perth...,08-OCT-2025,Canine astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,6478,"Mifsud,J.C.O., Harvey,E., Van Brussel,K., Olss...",NA,CaAstV/Dog/AUS/perth_6/2022,Australia: Perth,NA,Mar-2022,feces,Canis lupus familiaris,No,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y08628,Y08628.1,Human astrovirus type 2 gene encoding capsid p...,26-JUL-2016,Human astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,302,"Monceyron,C.",NA,sample 6190,NA,NA,NA,NA,NA,No,1
Y08627,Y08627.1,Human astrovirus type 1 gene encoding capsid p...,26-JUL-2016,Human astrovirus,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,NA,Astroviridae,300,"Monceyron,C.",NA,sample 7783,NA,NA,NA,NA,NA,No,1
Z66541,Z66541.1,Human astrovirus type 8 orf2 gene for capsid p...,30-OCT-1995,Human astrovirus 8,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,Mamastrovirus hominis,Astroviridae,2434,"Carter,M.J.",NA,NA,NA,NA,NA,NA,NA,No,1
Z46658,Z46658.1,Human astrovirus type 6 gene for capsid polypr...,14-NOV-1994,Human astrovirus 6,Viruses;Riboviria;Orthornavirae;Pisuviricota;S...,Mamastrovirus hominis,Astroviridae,2418,"Willcocks,M.M.",NA,NA,NA,NA,NA,NA,NA,No,1


In [ ]:
orf_coord("Astroviridae_15102025_template.gb", "ORF_names.csv")

In [5]:
outfile = "Astroviridae_15102025.gb"
outfile = "D:\Virology\RNA_viruses\Astroviridae_database\Astroviridae.gb"
print(os.path.abspath(outfile))

D:\Virology\RNA_viruses\Astroviridae_database\Astroviridae.gb


In [ ]:
input_file = "Astroviridae_15102025_template.gb"
data = fetch_metadata_from_gb(input_file)